**BRONZE LAYER - INGESTION**
- It will read the raw CSV file into a Spark DataFrame
- Keep schema as-is (messy values like ₹, %, commas remain)

In [0]:
df_bronze = spark.read.csv(
    "/Volumes/workspace/storage/s1/amazon.csv",
    header=True,
    inferSchema=True
)

display(df_bronze.limit(10))


product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹399,"₹1,099",64%,4.2,"24,269","High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to contact us.","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBBSNLYT3ONILA,AHCTC6ULH4XB6YHDY6PCH2R772LQ,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,AENGU523SXMOS7JPDTW52PNNVWGQ,AEQJHCVTNINBS4FKTBGQRQTGTE5Q,AFC3FFC5PKFF5PMA52S3VCHOZ5FQ","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1KD19VHEDV0OR,R3C02RMYQMK6FC,R39GQRVBUZBWGY,R2K9EDOE15QIRJ,R3OI7YT648TL8I","Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good","Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/51UsScvHQNL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1
B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be it an android smartphone (Mi, Samsung, Oppo, Vivo, Realme, OnePlus, etc), tablet, laptop (Macbook, Chromebook, etc)|Supports Quick Charging (2.0/3.0)|Unbreakable – Made of special braided outer with rugged interior bindings, it is ultra-durable cable that won’t be affected by daily rough usage|Ideal Length – It has ideal length of 1.5 meters which is neither too short like your typical 1meter cable or too long like a 2meters cable|Supports maximum 3A fast charging and 480 Mbps data transfer speed|6 months manufacturer warranty from the date of purchase","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBXNGXZJT525AQ,AHONIZU3ICIEHQIGQ6R2VFRSBXOQ,AFPHD2CRPDZMWMBL7WXRSVYWS5JA,AEZ346GX3HJ4O4XNRPHCNHXQURMQ,AEPSWFPNECKO34PUC7I56ITGXR6Q,AHWVEHR5DYLVFTO2KF3IZATFQSWQ,AH4QT33M55677I7ISQOAKEQWACYQ","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Placeholder,BharanI,sonia,Niam","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GW

**Silver Layer: Cleaning**
- It will remove unwanted symbols (₹, %, commas) from price fields
- It will convert discounted_price, actual_price, discount_percentage to DOUBLE
- It will convert rating to DOUBLE, rating_count to INT
- I used try_cast to handle malformed values (set them to NULL instead of failing)
- Saved as 'amazon_silver' Delta table

In [0]:

from pyspark.sql.functions import regexp_replace, expr

df_clean = df_bronze \
    .withColumn("discounted_price",
                expr("try_cast(regexp_replace(regexp_replace(discounted_price, '₹', ''), ',', '') as double)")) \
    .withColumn("actual_price",
                expr("try_cast(regexp_replace(regexp_replace(actual_price, '₹', ''), ',', '') as double)")) \
    .withColumn("discount_percentage",
                expr("try_cast(regexp_replace(discount_percentage, '%', '') as double)")) \
    .withColumn("rating", expr("try_cast(rating as double)")) \
    .withColumn("rating_count", expr("try_cast(rating_count as int)"))

# Save as Silver
df_clean.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("amazon_silver")

display(spark.table("amazon_silver").limit(10))


product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,399.0,1099.0,64.0,4.2,null,"High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to contact us.","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBBSNLYT3ONILA,AHCTC6ULH4XB6YHDY6PCH2R772LQ,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,AENGU523SXMOS7JPDTW52PNNVWGQ,AEQJHCVTNINBS4FKTBGQRQTGTE5Q,AFC3FFC5PKFF5PMA52S3VCHOZ5FQ","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1KD19VHEDV0OR,R3C02RMYQMK6FC,R39GQRVBUZBWGY,R2K9EDOE15QIRJ,R3OI7YT648TL8I","Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good","Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/51UsScvHQNL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1
B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,199.0,349.0,43.0,4.0,null,"Compatible with all Type C enabled devices, be it an android smartphone (Mi, Samsung, Oppo, Vivo, Realme, OnePlus, etc), tablet, laptop (Macbook, Chromebook, etc)|Supports Quick Charging (2.0/3.0)|Unbreakable – Made of special braided outer with rugged interior bindings, it is ultra-durable cable that won’t be affected by daily rough usage|Ideal Length – It has ideal length of 1.5 meters which is neither too short like your typical 1meter cable or too long like a 2meters cable|Supports maximum 3A fast charging and 480 Mbps data transfer speed|6 months manufacturer warranty from the date of purchase","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBXNGXZJT525AQ,AHONIZU3ICIEHQIGQ6R2VFRSBXOQ,AFPHD2CRPDZMWMBL7WXRSVYWS5JA,AEZ346GX3HJ4O4XNRPHCNHXQURMQ,AEPSWFPNECKO34PUC7I56ITGXR6Q,AHWVEHR5DYLVFTO2KF3IZATFQSWQ,AH4QT33M55677I7ISQOAKEQWACYQ","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Placeholder,BharanI,sonia,Niam","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY

**Monitoring: Null & Bad Data Checks**
- It counts NULL values in important columns (price, discount, rating, rating_count)
- It will check for negative or zero prices (invalid)
- Output shows data quality issues before moving to Gold

In [0]:

from pyspark.sql import functions as F

silver = spark.table("amazon_silver")

# Count nulls in important columns
null_counts = silver.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c + "_nulls")
    for c in ["discounted_price", "actual_price", "discount_percentage", "rating", "rating_count"]
])

display(null_counts)

# Check for negative or zero prices (invalid)
bad_prices = silver.filter((F.col("discounted_price") <= 0) | (F.col("actual_price") <= 0))
print("Bad price rows:", bad_prices.count())


discounted_price_nulls,actual_price_nulls,discount_percentage_nulls,rating_nulls,rating_count_nulls
57,54,57,58,1148


Bad price rows: 0


**Silver Cleanup: Filter Valid Rows**
- Load the Silver table
- Remove rows where discounted_price or actual_price is NULL
- Ensures only valid numeric rows are passed to feature engineering

In [0]:

silver = spark.table("amazon_silver")

# Keep only rows with valid prices
silver_clean = silver.filter(
    silver.discounted_price.isNotNull() & silver.actual_price.isNotNull()
)


**Feature Engineering: Popularity Score**
- Add a new column 'popularity_score'
- Formula = rating_count * rating
- Higher score = more popular product
- This feature will help us decide anchor products

In [0]:

from pyspark.sql import functions as F

silver_pop = silver_clean.withColumn(
    "popularity_score", 
    (F.col("rating_count") * F.col("rating"))
)


**Gold Layer: Suggested Pricing**

Step 1: Simplify category
 - Take only the first part of the category string (split by "|")
 
 Step 2: It define anchors vs non-anchors
- Anchors = top 10% most popular products (by popularity_score)
- Non-anchors = all other products

Step 3: Compute anchor_median_price
- Median discounted_price within each simplified category

Step 4: Compute suggested_price for non-anchors
- Formula: 90% anchor_median_price + 10% actual_price

Step 5: Save results as Gold table (amazon_gold_pricing)

In [0]:

from pyspark.sql import functions as F

# Simplify category: take the first part before the first "|"
silver_clean = silver_pop.withColumn(
    "category_simple",
    F.split(F.col("category"), r"\|").getItem(0)
)

# Recompute anchors and non-anchors with cleaned category
threshold = silver_clean.approxQuantile("popularity_score", [0.9], 0.01)[0]
anchors = silver_clean.filter(F.col("popularity_score") >= threshold)
non_anchors = silver_clean.filter(F.col("popularity_score") < threshold)

# Median price by cleaned category
anchor_prices = anchors.groupBy("category_simple").agg(
    F.percentile_approx("discounted_price", 0.5).alias("anchor_median_price")
)

# Join and compute suggested price
suggested = non_anchors.join(anchor_prices, "category_simple", "left") \
    .withColumn(
        "suggested_price",
        F.round((F.col("anchor_median_price") * 0.9) + (F.col("actual_price") * 0.1), 2)
    )

# Save Gold table
suggested.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("default.amazon_gold_pricing")

display(spark.table("default.amazon_gold_pricing").limit(10))


category_simple,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link,popularity_score,anchor_median_price,suggested_price
Home&Kitchen,B09KNMLH4Y,"R B Nova Lint/Fabric Shaver for Cloths, Lint Remover for Woolen Sweaters, Blankets, Jackets/Burr Remover Pill Remover from Carpets, Pack of 1","Home&Kitchen|Kitchen&HomeAppliances|Vacuum,Cleaning&Ironing|Irons,Steamers&Accessories|LintShavers",398.0,1999.0,80.0,4.1,257,"Quick and efficient Switch to turn it on and gently run the shaving head over the surface of the clothes.|Corded The lint remover runs on an inbuilt battery and has a corded wire. Battery operated lint removers run out of power after a few times of use but you save hundreds on batteries with this corded lint remover.|Dual protection The adjustable height spacer protects your garments from being cut or snagged or caught. It also protects and reinforces your hands from accidentally touching the shaver foil. A cleaning brush is included in the pack to help you easily clean your appliance after use.|Smart design With large shaving head, detachable lint catcher and comfort grip handle. Slide downward to release the lint catcher gently to remove and clean the collected lint. The comfort grip handle allows the lint remover to fit comfortably in your hand, giving you a good and control as you de-fuzz your garments|Easily remove fabric fluff, fuzz, bobbles, pills, burr or roye from all types of garments and fabrics","AHNVMNUO3GZIOGQKKAGSPTXY5VEQ,AFKDML4DUIIFJPCMTZZMJTLQR5VQ,AFWWEYDN3ERGL6UJNV7GF6PB66JA,AFNBPDYECMALHAA3NL6L3JUUAWBQ,AFQ7CJTBPHCZUKFO6G7Y33VCXLKA,AGLQVPXIXE7WNSV7S5QV5UCNQUZQ,AGYVZCXMAA6OB3UGJXCUBQCQCYRQ,AECLL66KOCXSQJ2GAS5FF2DEIXQA","Amazon Customer,Mannat s.,Tanveer,Amit arora,girish saini,lokesh vijrai,Vipin,sadiya swaleh","R27SHBAT3K3F1R,R3EMA46KP56OXK,R2D7V4YKNKCXD4,R3UHV5AN1DF5H3,RV77H2T0BJN4V,R3O7GL8KXFAPBF,R2HXBI1ECJPV3J,R2QICML7QBXEC0","The Best Purchase,Very useful product,Good products,Very handy to remove lint,Good product,Good product,Good,It's worth it.","I've been an Amazon customer since 2014/15. This is, without any doubt the best purchase I've ever made.If you have polyester, polyester+cotton, acrylic, acrylic+wool or acrylic+polyester clothes you know that after sometime (maybe years, could be months) they will start to pill. Pilling is those small round blobs on the cloth: make the item look old and haggard, as if they've hand me downs.Well, this product will reset it back to its original look and feel. It's crazy how good it works, on socks, sweaters, mufflers, caps, gloves, fleece jackets, fleece pants, anything that has artificial fibre composition.Don't but the one with battery because you'll be restricted to the capacity of the battery and when it degrades the product is useless - corded is the way to go for me.Five stars without a doubt.,Easy to use and very useful product.,I am very happy,Made Plastic could have been better,Good product ,Works really well...,Removes lint in one go. Good to use. First time user.,Good,Very easy to use and it works really well. Leaves a near nd clean fabric after removing the lint completely.",https://m.media-amazon.com/images/I/51oPN7WqUwL._SY300_SX300_QL70_FMwebp_.jpg,https://www.amazon.in/Remover-Sweaters-Blankets-Jackets-Carpets/dp/B09KNMLH4Y/ref=sr_1_37_mod_primary_new?qid=1672923592&s=kitchen&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-37,1053.6999999999998,1999.0,1999.0
Home&Kitchen,B0B8XNPQPN,"Pigeon Healthifry Digital Air Fryer, 360° High Speed Air Circulation Technology 1200 W with Non-Stick 4.2 L Basket - Green",Home&Kitchen|Kitchen&HomeAppliances|SmallKitchenAppliances|DeepFatFryers|AirFryers,3599.0,7950.0,55.0,4.2,136,"Only Air Fryer with Appointment Function used to make fried foods such as meat, pastries, and potato chips.|AIR FRYER CAPACITY: 4.2 litre capacity is enough to enjoy your favorite French fr

**Serving Layer: Comparison View**

- Load Gold table (amazon_gold_pricing)
- Select key fields for demo:
category_simple, product_id, product_name,
  actual_price, anchor_median_price, suggested_price
- Order by category and show first 20 rows

In [0]:

# Load the Gold table
gold = spark.table("default.amazon_gold_pricing")

# Select useful columns for demo
comparison = gold.select(
    "category_simple",
    "product_id",
    "product_name",
    "actual_price",
    "anchor_median_price",
    "suggested_price"
).orderBy("category_simple").limit(20)

display(comparison)


category_simple,product_id,product_name,actual_price,anchor_median_price,suggested_price
Computers&Accessories,B09C6H53KH,"Duracell Type-C To Micro 1.2M braided Sync & Charge Cable, USB C to Micro Fast Charge Compatible for fast data transmission (Black)",699.0,389.0,420.0
Computers&Accessories,B09X79PP8F,"MI 2-in-1 USB Type C Cable (Micro USB to Type C) 30cm for Smartphone, Headphone, Laptop (White)",299.0,389.0,380.0
Computers&Accessories,B09Q8HMKZX,"Portronics Konnect L 20W PD Quick Charge Type-C to 8-Pin USB Mobile Charging Cable, 1.2M, Tangle Resistant, Fast Data Sync(Grey)",699.0,389.0,420.0
Computers&Accessories,B0BP7XLX48,"Syncwire LTG to USB Cable for Fast Charging Compatible with Phone 5/ 5C/ 5S/ 6/ 6S/ 7/8/ X/XR/XS Max/ 11/12/ 13 Series and Pad Air/Mini, Pod & Other Devices (1.1 Meter, White)",1999.0,389.0,550.0
Computers&Accessories,B09CMP1SC8,"Ambrane 2 in 1 Type-C & Micro USB Cable with 60W / 3A Fast Charging, 480 mbps High Data, PD Technology & Quick Charge 3.0, Compatible with All Type-C & Micro USB Devices (ABDC-10, Black)",499.0,389.0,400.0
Computers&Accessories,B0B21C4BMX,"Portronics Konnect Spydr 31 3-in-1 Multi Functional Cable with 3.0A Output, Tangle Resistant, 1.2M Length, Nylon Braided(Zebra)",899.0,389.0,440.0
Computers&Accessories,B09RZS1NQT,"Sounce 65W OnePlus Dash Warp Charge Cable, 6.5A Type-C to USB C PD Data Sync Fast Charging Cable Compatible with One Plus 8T/ 9/ 9R/ 9 pro/ 9RT/ 10R/ Nord & for All Type C Devices – Red, 1 Meter",999.0,389.0,450.0
Computers&Accessories,B0B4DT8MKT,"Wecool Unbreakable 3 in 1 Charging Cable with 3A Speed, Fast Charging Multi Purpose Cable 1.25 Mtr Long, Type C cable, Micro Usb Cable and Cable for iPhone, White",1499.0,389.0,500.0
Computers&Accessories,B084N18QZY,"Belkin USB C to USB-C Fast Charging Type C Cable, 60W PD, 3.3 feet (1 meter) for Laptop, Personal Computer, Tablet, Smartphone - Black, USB-IF Certified",849.0,389.0,435.0
Computers&Accessories,B0B86CDHL1,"oraimo 65W Type C to C Fast Charging Cable USB C to USB C Cable High Speed Syncing, Nylon Braided 1M length with LED Indicator Compatible For Laptop, Macbook, Samsung Galaxy S22 S20 S10 S20Fe S21 S21 Ultra A70 A51 A71 A50S M31 M51 M31S M53 5G",899.0,389.0,440.0
